## Geospatial Unit Data Cleaning for Measuring Regional Economic Inequality in China

- **Date**: February 19, 2024
- **Author**: Xiaozhong Sun
- **Abstract**: This code notebook is used to clean provincial-level economic data for measuring regional inequality in China from 1952 to 2022. The analysis will focus on three regional components:

  - Between Provinces
  - Coastal and Inland
  - Different Economic Regions

  There will be four dimensions for inequality measures:

    - GDP and by industrial sectors
    - International Trade
    - Fiscal spending and revenue

---

### Import Packages

In [1]:
import ee
import geemap
import os
import glob
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
geemap.ee_initialize()

### Data Preparation

#### Step 1: Reading Excel Files

In [65]:
# Path to the directory containing the excel files
data_directory_1 = "./data/raw_data/census_data_by_province"

# Get a list of all Excel files in the directory
province_files = glob.glob(os.path.join(data_directory_1, "prov_*.xlsx"))

# Initialize an empty DataFrame to store all data
all_data = []

for file in province_files:
    # Load the dataset
    df = pd.read_excel(file, header=[0, 1], index_col=0)
    
    # Simplify MultiIndex headers and prepare the DataFrame
    df.columns = [f'{y[0]} {y[1]}' for y in df.columns]
    df.reset_index(inplace=True)
    df.rename(columns={df.columns[0]: 'Province'}, inplace=True)
    
    # Append the DataFrame to the list
    all_data.append(df)

# Concatenate all the DataFrames in the list into one DataFrame
combined_df = pd.concat(all_data, ignore_index=True)

# Melt the DataFrame
all_data_long = pd.melt(combined_df, id_vars=['Province'], var_name='Year_Variable', value_name='Value')

# Extract 'Year' and 'Variable' from 'Year_Variable'
all_data_long['Year'] = all_data_long['Year_Variable'].apply(lambda x: x.split(' ')[0])
all_data_long['Variable'] = all_data_long['Year_Variable'].apply(lambda x: ' '.join(x.split(' ')[1:]))
all_data_long.drop('Year_Variable', axis=1, inplace=True)

# Pivot the DataFrame
all_data_pivoted = all_data_long.pivot_table(index=['Province', 'Year'], columns='Variable', values='Value', aggfunc='first').reset_index()
all_data_pivoted.columns.name = None  # Remove the categorization name to flatten the DataFrame properly

# Optional: You may want to sort the combined DataFrame based on 'Province' and 'Year' or any other criteria
all_data_pivoted.sort_values(by=['Province', 'Year'], inplace=True)

# Optional: Reset index if you want the index to start from 0
all_data_pivoted.reset_index(drop=True, inplace=True)

# Display or save the combined DataFrame
print(all_data_pivoted.head(1000))


/Users/xiaozhongsun/miniconda3/envs/geeineq/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/xiaozhongsun/miniconda3/envs/geeineq/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/xiaozhongsun/miniconda3/envs/geeineq/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/xiaozhongsun/miniconda3/envs/geeineq/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl'

    Province   Year GDP(亿元) 人均GDP(元) 农业人口数(万人) 农民最终消费支出(亿元) 出口总额(万美元)  \
0        上海市  1949年   20.28      274     35.17         1.52      8200   
1        上海市  1950年   22.43      309     35.38         1.74     10400   
2        上海市  1951年   31.52      388     35.75         2.05     10000   
3        上海市  1952年   36.66      430     36.55         2.28     11900   
4        上海市  1953年   51.71      590     38.09         2.42     18300   
..       ...    ...     ...      ...       ...          ...       ...   
995       新疆  1982年   65.24    488.0    921.83           --      9032   
996       新疆  1983年    79.0    583.0    927.44           --      9697   
997       新疆  1984年    90.0    661.0    919.84           --     15966   
998       新疆  1985年  112.24    820.0    922.47        26.41     18020   
999       新疆  1986年  129.04    924.0    933.89           --     20535   

    固定资产投资额(万元) 城镇居民最终消费支出(亿元) 总人口数(万人) 第一产业增加值(亿元) 第三产业增加值(亿元) 第二产业增加值(亿元)  \
0          1100          10.73   502.92     

In [66]:
# Define a dictionary to map Chinese column names to English
column_name_mapping = {
      'Province': 'province',
      'Year': 'year',
      'GDP(亿元)': 'GDP (100million)',
      '人均GDP(元)': 'GDP per capita',
      '第一产业增加值(亿元)': 'Primary industry added value (100million)',
      '第二产业增加值(亿元)': 'Secondary industry added value (100million)',
      '第三产业增加值(亿元)': 'Tertiary industry added value (100million)',
      '农民最终消费支出(亿元)': 'Rural final consumption expenditure (100million)',
      '城镇居民最终消费支出(亿元)': 'Urban residents final consumption expenditure (100million)',
      '总人口数(万人)': 'Total population (10thousand)',
      '非农业人口数(万人)': 'Non-agricultural population (10thousand)',
      '农业人口数(万人)': 'Agricultural population (10thousand)',
      '财政支出(万元)': 'Fiscal expenditure (10000)',
      '财政收入(万元)': 'Fiscal revenue (10000)',
      '固定资产投资额(万元)': 'Fixed asset investment (10000)',
      '出口总额(万美元)': 'Total export (10000 USD)',
      '进口总额(万美元)': 'Total import (10000 USD)',
      '进出口总额(万美元)': 'Total import and export (10000 USD)'
}

# Rename the columns from Chinese to English
all_data_pivoted.rename(columns=column_name_mapping, inplace=True)

# Replace '--' with NaN
all_data_pivoted.replace('--', np.nan, inplace=True)

# Display or save the combined DataFrame
print(all_data_pivoted.head(1000))


    province   year GDP (100million) GDP per capita  \
0        上海市  1949年            20.28            274   
1        上海市  1950年            22.43            309   
2        上海市  1951年            31.52            388   
3        上海市  1952年            36.66            430   
4        上海市  1953年            51.71            590   
..       ...    ...              ...            ...   
995       新疆  1982年            65.24          488.0   
996       新疆  1983年             79.0          583.0   
997       新疆  1984年             90.0          661.0   
998       新疆  1985年           112.24          820.0   
999       新疆  1986年           129.04          924.0   

    Agricultural population (10thousand)  \
0                                  35.17   
1                                  35.38   
2                                  35.75   
3                                  36.55   
4                                  38.09   
..                                   ...   
995                            

In [67]:
# Export the DataFrame to a CSV file
all_data_pivoted.to_csv('transformed_data.csv', index=False)